In [8]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    
    # Add the number of dB to each column name for each .csv file

    df_6dB.columns = [str(col) + '_6dB' for col in df_6dB.columns]
    df_0dB.columns = [str(col) + '_0dB' for col in df_0dB.columns]
    df_min6dB.columns = [str(col) + '_-6dB' for col in df_min6dB.columns]
    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,how='outer'), data_frames)
    
    df_merged.head()
    
    return df_merged

In [10]:
pump = merge_pd('pump')

In [11]:
pump.drop(columns=['normal(0)/abnormal(1)_6dB','normal(0)/abnormal(1)_0dB'],axis=1,inplace=True)
pump.rename(columns={'normal(0)/abnormal(1)_-6dB': 'normal(0)/abnormal(1)'}, inplace=True)

In [12]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [13]:
# 20% of the overal data will seperated for later validation of the model

X_model, X_valid = train_test_split(pump,test_size=0.2,random_state = 42)

y_valid = X_valid['normal(0)/abnormal(1)']
X_valid = X_valid.drop(columns=['normal(0)/abnormal(1)'])

# 64% of the overal data (80% of X_model, y_model) will be used to create a training set for the model
# 16% of the overal data (20% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model.drop(columns=['normal(0)/abnormal(1)'], axis=1),
                                                   X_model['normal(0)/abnormal(1)'],
                                                   test_size = 0.25,
                                                   random_state = 10)

In [16]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(RandomForestClassifier(class_weight='balanced'))
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv=5)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_sum_6dB', 'spectral_bandwidth_6dB',
       'zero_crossing_rate_6dB', 'melspectrogram_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[733  16]
 [ 22  70]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       749
           1       0.81      0.76      0.79        92

    accuracy                           0.95       841
   macro avg       0.89      0.87      0.88       841
weighted avg       0.95      0.95      0.95       841

accuracy score
0.9548156956004756
----------------------------------------------
TRAIN-VALIDATION
-------------

Selected Feature 6
Index(['melspectrogram_sum_6dB', 'rms_6dB', 'spectral_bandwidth_6dB',
       'zero_crossing_rate_6dB', 'max perc_6dB', 'melspectrogram_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[737  12]
 [ 17  75]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       749
           1       0.86      0.82      0.84        92

    accuracy                           0.97       841
   macro avg       0.92      0.90      0.91       841
weighted avg       0.96      0.97      0.97       841

accuracy score
0.9655172413793104
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[736  11]
 [ 12  82]]
classification report
              precision    recall  f1-score   support

          

Selected Feature 8
Index(['melspectrogram_max_6dB', 'melspectrogram_sum_6dB', 'rms_6dB',
       'spectral_bandwidth_6dB', 'spectral_flatness _6dB',
       'spectral_rolloff_6dB', 'zero_crossing_rate_6dB',
       'melspectrogram_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[736  13]
 [ 17  75]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       749
           1       0.85      0.82      0.83        92

    accuracy                           0.96       841
   macro avg       0.91      0.90      0.91       841
weighted avg       0.96      0.96      0.96       841

accuracy score
0.9643281807372176
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[740   7]
 [ 16  78]]
classificatio

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[749   0]
 [ 19  73]]
classification report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       749
           1       1.00      0.79      0.88        92

    accuracy                           0.98       841
   macro avg       0.99      0.90      0.94       841
weighted avg       0.98      0.98      0.98       841

accuracy score
0.9774078478002378
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[746   1]
 [ 15  79]]
classification report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       747
           1       0.99      0.84      0.91        94

    accuracy                           

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[744   5]
 [ 29  63]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       749
           1       0.93      0.68      0.79        92

    accuracy                           0.96       841
   macro avg       0.94      0.84      0.88       841
weighted avg       0.96      0.96      0.96       841

accuracy score
0.9595719381688466
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier()
[[747   0]
 [ 19  75]]
classification report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       747
           1       1.00      0.80      0.89        94

    accuracy                           0.98       841
   macro avg       0.99      0.

...

Selected Feature 14
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'spectral_bandwidth_6dB', 'spectral_rolloff_6dB',
       'zero_crossing_rate_6dB', 'max perc_6dB', 'melspectrogram_0dB',
       'melspectrogram_sum_0dB', 'max perc_0dB', 'melspectrogram_-6dB',
       'melspectrogram_sum_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[749   0]
 [ 20  72]]
classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       749
           1       1.00      0.78      0.88        92

    accuracy                           0.98       841
   macro avg       0.99      0.89      0.93       841
weighted avg       0.98      0.98      0.97       841

accuracy score
0.976218787158145
-----------------------

Selected Feature 16
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'spectral_bandwidth_6dB', 'spectral_flatness _6dB',
       'spectral_rolloff_6dB', 'zero_crossing_rate_6dB', 'max harm_6dB',
       'max perc_6dB', 'melspectrogram_0dB', 'melspectrogram_sum_0dB',
       'melspectrogram_-6dB', 'melspectrogram_sum_-6dB', 'rms_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[746   3]
 [ 22  70]]
classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       749
           1       0.96      0.76      0.85        92

    accuracy                           0.97       841
   macro avg       0.97      0.88      0.92       841
weighted avg       0.97      0.97      0.97       841

accuracy score
0.9702734839476813
-------

Selected Feature 18
Index(['melspectrogram_max_6dB', 'melspectrogram_sum_6dB', 'mfcc_6dB',
       'rms_6dB', 'spectral_bandwidth_6dB', 'spectral_flatness _6dB',
       'spectral_rolloff_6dB', 'zero_crossing_rate_6dB', 'max harm_6dB',
       'max perc_6dB', 'mfcc_0dB', 'mean harm_0dB', 'melspectrogram_-6dB',
       'melspectrogram_max_-6dB', 'melspectrogram_std_-6dB',
       'spectral_bandwidth_-6dB', 'spectral_rolloff_-6dB', 'max perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[732  17]
 [ 20  72]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       749
           1       0.81      0.78      0.80        92

    accuracy                           0.96       841
   macro avg       0.89      0.88      0.89       841
weighted avg       0.96      0.96      0.96       841

accuracy sco

Selected Feature 19
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'spectral_bandwidth_6dB', 'spectral_flatness _6dB',
       'spectral_rolloff_6dB', 'zero_crossing_rate_6dB', 'max perc_6dB',
       'min harm_6dB', 'min perc_6dB', 'melspectrogram_0dB',
       'melspectrogram_sum_0dB', 'spectral_bandwidth_0dB', 'max perc_0dB',
       'melspectrogram_-6dB', 'melspectrogram_sum_-6dB', 'rms_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[749   0]
 [ 21  71]]
classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       749
           1       1.00      0.77      0.87        92

    accuracy                           0.98       841
   macro avg       0.99      0.89      0.93       841
weighted

Selected Feature 21
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'spectral_centroid_6dB', 'spectral_bandwidth_6dB',
       'spectral_flatness _6dB', 'spectral_rolloff_6dB',
       'zero_crossing_rate_6dB', 'max harm_6dB', 'max perc_6dB',
       'min harm_6dB', 'melspectrogram_0dB', 'melspectrogram_sum_0dB',
       'melspectrogram_std_0dB', 'rms_0dB', 'melspectrogram_-6dB',
       'melspectrogram_sum_-6dB', 'melspectrogram_std_-6dB', 'rms_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[747   2]
 [ 22  70]]
classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       749
           1       0.97      0.76      0.85        92

    accuracy                           0.97       841
   macro avg       0.97      0.